# Side Concordance Analysis

This notebook assesses the effectiveness of pre-operative imaging by comparing imaging results to surgical findings. It evaluates whether each imaging modality accurately identifies surgical targets either exactly or by side, providing insights into the diagnostic value of each modality in surgical planning.

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 


preop_imaging_df = pd.read_excel('data/final_cleaned_with_concordance.xlsx')

In [3]:
modalities = ['Ultrasound', 'SPECT/CT', '4D CT Scan', 'Sestamibi', 'MRI']

Define helper functions:
- `extract_side`: Extracts side (Left/Right) from location strings
- `check_concordance_exact_or_side`: Determines concordance between imaging and surgical findings

In [4]:
def extract_side(location):
    
    # return 'No Scan' if location is missing
    if pd.isnull(location):
        return 'No Scan'
    
    # return 'Left' if location mentions 'Left'
    elif 'Left' in location:
        return 'Left'
    
    # return 'Right' if location mentions 'Right'
    elif 'Right' in location:
        return 'Right'
    

In [5]:
def check_concordance_exact_or_side(row, modality_col):
    
    # If imaging result is missing
    if pd.isnull(row[modality_col]):
        return 'No Scan'
    
    # If imaging result matches surgical finding exactly
    elif row[modality_col] == row['Surgical Findings']:
        return 'Yes (Exact)'
    
    # If imaging side matches surgical side
    elif extract_side(row[modality_col]) == extract_side(row['Surgical Findings']):
        return 'Yes (Side)'
    
    # Otherwise, not concordant
    else:
        return 'No'

Apply concordance checks across all imaging modalities and create new labeled columns in the dataframe.


In [6]:
# iterate through the list of imaging modalities 
for modality in modalities:

# Create a new column with concordance results for that modality
    preop_imaging_df[modality + '_Concordance'] = preop_imaging_df.apply(
        lambda row: check_concordance_exact_or_side(row, modality),
        axis=1
    )
    

In [7]:
# checking to make sure it worked 
for modality in modalities:
    display(preop_imaging_df[[modality, 'Surgical Findings', modality + '_Concordance']].head())

,Ultrasound,Surgical Findings,Ultrasound_Concordance
0,NaN,Right Inferior,No Scan
1,Non-localizing,Left Inferior,No
2,Left Inferior,Left Inferior,Yes (Exact)
3,NaN,"Left Superior, Right Superior",No Scan
4,Left,Left Superior,Yes (Side)


,SPECT/CT,Surgical Findings,SPECT/CT_Concordance
0,NaN,Right Inferior,No Scan
1,NaN,Left Inferior,No Scan
2,NaN,Left Inferior,No Scan
3,NaN,"Left Superior, Right Superior",No Scan
4,NaN,Left Superior,No Scan


,4D CT Scan,Surgical Findings,4D CT Scan_Concordance
0,NaN,Right Inferior,No Scan
1,NaN,Left Inferior,No Scan
2,NaN,Left Inferior,No Scan
3,NaN,"Left Superior, Right Superior",No Scan
4,NaN,Left Superior,No Scan


,Sestamibi,Surgical Findings,Sestamibi_Concordance
0,Right Inferior,Right Inferior,Yes (Exact)
1,Non-localizing,Left Inferior,No
2,Left Inferior,Left Inferior,Yes (Exact)
3,Left Inferior,"Left Superior, Right Superior",Yes (Side)
4,Left,Left Superior,Yes (Side)


,MRI,Surgical Findings,MRI_Concordance
0,Right Inferior,Right Inferior,Yes (Exact)
1,Right Inferior,Left Inferior,No
2,NaN,Left Inferior,No Scan
3,Non-localizing,"Left Superior, Right Superior",No
4,NaN,Left Superior,No Scan


In [8]:
preop_imaging_df.to_excel('side_concordance_results.xlsx', index=False)